In [1]:
# Nanodegree "Machine Learning Engineer with Microsoft Azure"
# Project 1 - solution created by Marco Stolpe
#
# The following code is based on the skeleton provided in the
# context of the project, on the nanodegree's lectures, and
# on the Azure ML and Scikit-learn SDK / class documentation.
#
# The way the data is split and the best hyperdrive model is trained
# are based on ideas presented in
#
# https://stats.stackexchange.com/questions/11602/training-on-the-full-dataset-after-cross-validation
#
# Storing the AutoML model requires an upgrade of the Azure ML SDK
# libraries before running the notebook and can be done in a terminal
# on the compute instance running Jupyter with the following command:
#
# pip install --upgrade azureml-sdk[automl]
#
# This information was taken from the knowledge base article
#
# https://knowledge.udacity.com/questions/733910
#

from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-189314
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-189314


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create a compute cluster "ndeg-prj1-clust" with a maximum of four
# nodes for all subsequent experiments.

cluster_name = "ndeg-prj1-clust"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
    
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    '--C': uniform(0.01, 1.0),  # inverse of regularization strength - low values mean higher regularization
    '--max_iter': choice(100, 200, 500, 1000)  # maximum number of iterations for the solver
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1) # evaluate performance every two runs,
                                                               # stop if lower than 1% point difference to
                                                               # best result in previous two runs

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="train.py",
    compute_target=aml_compute, # use the previously created compute cluster
    environment=sklearn_env
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name='accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_520e61ee-8d64-4b53-9b61-43cf38576353
Web View: https://ml.azure.com/runs/HD_520e61ee-8d64-4b53-9b61-43cf38576353?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-189314/workspaces/quick-starts-ws-189314&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-03-18T13:11:13.385674][API][INFO]Experiment created<END>\n""<START>[2022-03-18T13:11:14.297632][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-03-18T13:11:15.032403][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_520e61ee-8d64-4b53-9b61-43cf38576353
Web View: https://ml.azure.com/runs/HD_520e61ee-8d64-4b53-9b61-43cf38576353?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-189314/workspaces/quick-starts-ws-189314&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_520e61ee-8d64-4b53-9b61-43cf38576353',
 'target': 'ndeg-prj1-clust',
 'status': 'Completed',
 'startTimeUtc': '2022-03-18T13:11:13.075021Z',
 'endTimeUtc': '2022-03-18T13:33:41.964184Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '448c42ac-b525-49fd-a839-a9ce25753467',
  'user_agent': 'python/3.8.1 (Linux-5.4.0-1068-azure-x86_64-with-glibc2.10) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.39.0',
  'space_size': 'infinite_space_size',
  'score': '0.9133156297420334',
  'best_child_run_id': 'HD_520e61ee-8d64-4b53-9b61-43cf38576353_15',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg189314.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_520e61ee-8d64-4b53-9b61-43cf3857

In [5]:
import joblib
# Get your best run and save the model from that run.
# Also show the primary metric for the best model found and
# display the hyperparameters of the best model.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best run ID:', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])
print('Parameters:', parameter_values)

best_run.download_file('outputs/model.pkl', './outputs/best_model_hyperdrive.pkl', _validate_checksum=True)

Best run ID: HD_520e61ee-8d64-4b53-9b61-43cf38576353_15
Accuracy: 0.9133156297420334
Parameters: ['--C', '0.3538862052764436', '--max_iter', '500']


In [6]:
# The data preparation for the AutoML run needed to be adjusted, as providing
# a pandas.DataFrame as training set in the AutoMLConfig lead to an error
# message. The solution in the following stores the data as a tabular dataset
# in the workspace's blob storage instead or loads it if it was already stored,
# which works with AutoML. 

from azureml.exceptions import UserErrorException
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Datastore, Dataset

ds = None

try:
    ds = Dataset.get_by_name(ws, 'bankmarketing-train')
    
except UserErrorException:
    table_ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
    x, y = clean_data(table_ds)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=20, stratify=y)
    x_train = x_train.assign(Label = y_train)
    x_test = x_test.assign(Label = y_test)
    datastore = Datastore.get(ws, 'workspaceblobstore')
    ds = Dataset.Tabular.register_pandas_dataframe(x_train, datastore, 'bankmarketing-train', show_progress=True)
    ds_test = Dataset.Tabular.register_pandas_dataframe(x_test, datastore, 'bankmarketing-test', show_progress=True)
    

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/9d91d084-6f09-44d6-b325-e9fd83f6effa/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.
Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/6466129d-01ee-416a-9257-f76661e0e731/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [7]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='Label',
    n_cross_validations=5,
    compute_target=aml_compute)

In [8]:
from azureml.widgets import RunDetails

automl_run = exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_948fdadf-9c6c-4912-b21a-9f97534df843,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_948fdadf-9c6c-4912-b21a-9f97534df843,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Nu

{'runId': 'AutoML_948fdadf-9c6c-4912-b21a-9f97534df843',
 'target': 'ndeg-prj1-clust',
 'status': 'Completed',
 'startTimeUtc': '2022-03-18T13:34:41.014793Z',
 'endTimeUtc': '2022-03-18T14:17:15.735483Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ndeg-prj1-clust',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-189314","workspace_name":"quick-starts-ws-189314","region":"southcentralus","compute_target":"ndeg-prj1-clust","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_bat

In [9]:
import joblib

# Note that the following step requires upgrading the AzureML SDK on the client
best_run, best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()

print('Best run ID:', best_run.id)
print('Accuracy:', best_run_metrics['accuracy'])

joblib.dump(best_model, './outputs/best_model_automl.pkl')

Best run ID: AutoML_948fdadf-9c6c-4912-b21a-9f97534df843_37
Accuracy: 0.917412746585736


['./outputs/best_model_automl.pkl']

In [10]:
# The following code can be used to inspect the best model found by AutoML
for step in best_model.steps:
    print(step)

('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification'))
('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('29', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.625, reg_lambda=0.8333333333333334, subsample=0.8, tree_method='auto'))], verbose=False)), ('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGB

In [11]:
# Finally, the created cluster should be deleted
aml_compute.delete()